In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
from PIL import Image
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import cv2
import os
import glob
import pickle

Using TensorFlow backend.


In [3]:
filename = "flickr/Flickr8k_text/Flickr8k.token.txt"
file = open(filename, 'r')
doc = file.read()
file.close()
count = 0
for line in doc.split('\n'):
    count = count + 1
print(count)
print(doc[:400])

40461
1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg#3	A little girl climbing the stairs to her playhouse .
1000268201_693b08cb0e.jpg#4	A little girl in a pink dress going into a wood


In [4]:
def load_descriptions(doc):
    mapping = dict()
    for line in doc.split('\n'):
        tokens = line.split()
        if len(line) < 2:
            continue
        image_id = tokens[0]
        image_desc = tokens[1:]
        image_desc = ' '.join(image_desc)
        image_id = image_id.split('.')[0]
        if image_id in mapping:
            mapping[image_id].append(image_desc)
        if image_id not in mapping:
            mapping[image_id] = list()
            mapping[image_id].append(image_desc)
    return mapping
descriptions = load_descriptions(doc)
print('loaded=', len(descriptions))

loaded= 8092


In [5]:
#print(descriptions)
print(list(descriptions.keys())[:5])

['3432637363_3ba357e2da', '411175971_0fffd3b8c6', '3527184455_1a9c074ff2', '540604040_bec822c144', '3106791484_13e18c33d8']


In [6]:
print(len(descriptions))
print(descriptions['1000268201_693b08cb0e'])
print(descriptions['3712923460_1b20ebb131'])

8092
['A child in a pink dress is climbing up a set of stairs in an entry way .', 'A girl going into a wooden building .', 'A little girl climbing into a wooden playhouse .', 'A little girl climbing the stairs to her playhouse .', 'A little girl in a pink dress going into a wooden cabin .']
['a bunch of people in camo pants run .', 'A group of men and woman wearing army pants and a white shirt are running .', 'A group of men in camouflage pants and white t-shirts are running in a drill yard .', 'A group of people in army pants running .', 'A group of people wearing army clothes run together down the street .']


In [7]:
table = str.maketrans('', '', string.punctuation)
for key, desc_list in descriptions.items():
    #print(key, desc_list)
    for i in range(len(desc_list)):
        #print('1',desc_list[i])
        desc = desc_list[i]
        #print('2',desc)
        desc = desc.split()
        #print('3',desc)
        desc = [word.lower() for word in desc]
        #print('4',desc)
        desc = [w.translate(table) for w in desc]
        #print('5',desc)
        desc = [word for word in desc if (len(word) > 1)]
        #print('6',desc)
        desc = [word for word in desc if word.isalpha()]
        #print('7',desc)
        desc_list[i] = ' '.join(desc)
        #print('8',desc_list)
#print(desc_list)
#print(len(descriptions))
print(len(descriptions.keys()))

8092


In [8]:
print(descriptions['3712923460_1b20ebb131'])

['bunch of people in camo pants run', 'group of men and woman wearing army pants and white shirt are running', 'group of men in camouflage pants and white tshirts are running in drill yard', 'group of people in army pants running', 'group of people wearing army clothes run together down the street']


TypeError: list indices must be integers or slices, not str

In [9]:
vocabulary = set()
for key in descriptions.keys():
    [vocabulary.update(d.split()) for d in descriptions[key]]
print('size = ', len(vocabulary))
print(len(descriptions))
print(len(descriptions.keys()))

size =  8763
8092
8092


In [10]:
train_captions = []
for key,val in descriptions.items():
    for captions in val:
        train_captions.append(captions)

wordcount_max = 10
word_count = {}
nsents = 0
for sent in train_captions:
    nsents = nsents + 1
    for w in sent.split():
        word_count[w] = word_count.get(w, 0) + 1
vocab = [w for w in word_count if word_count[w] > wordcount_max]
print('words = ', len(vocab))


words =  1839


In [11]:
print(vocab)

['leafy', 'overhead', 'during', 'attempts', 'fountains', 'read', 'dive', 'guard', 'sun', 'looks', 'wild', 'desk', 'bull', 'autumn', 'palm', 'jockeys', 'singing', 'blurry', 'swim', 'floral', 'court', 'market', 'tackling', 'pointing', 'uniformed', 'cheer', 'stops', 'midair', 'lot', 'hand', 'hikes', 'friend', 'together', 'neighborhood', 'who', 'tiny', 'part', 'own', 'fuzzy', 'stunt', 'merrygoround', 'try', 'comes', 'jump', 'without', 'basketball', 'marked', 'trunk', 'pen', 'hair', 'arms', 'squirted', 'mohawk', 'military', 'shoot', 'furry', 'plays', 'sit', 'yard', 'smaller', 'balls', 'crying', 'sofa', 'elderly', 'surrounded', 'two', 'goalie', 'leaps', 'wave', 'toilet', 'toy', 'wood', 'go', 'bounds', 'plane', 'holds', 'bow', 'couple', 'there', 'very', 'skirt', 'sticking', 'fur', 'friends', 'cat', 'racing', 'grocery', 'closeup', 'goat', 'case', 'rock', 'teammate', 'old', 'microphone', 'cloudy', 'blow', 'pass', 'flat', 'uses', 'toys', 'harness', 'dusty', 'hillside', 'beside', 'identical', 'fi

In [12]:
############### already done. Dont execute again please #######################
f = open("flickr/Flickr8k_Dataset/vocabulary.txt", "a+")
for i in range(len(vocab)):
    f.write(vocab[i])
    f.write("\n")
f.close()

In [13]:
file = open('flickr/Flickr8k_text/Flickr_8k.trainImages.txt')
doc = file.read()
train_images = list()
for word in doc.split('\n'):
    image_identifier = word.split('.')[0]
    train_images.append(image_identifier)
print(len(train_images))

6001


In [15]:
train_descriptions = dict()
#train_descriptions_intermediate = list()
for line in str(descriptions).split('\n'):
    for image_id in descriptions:
        train_descriptions_intermediate = list()
        for i in range(len(descriptions[image_id])):
            train_descriptions_intermediate.append('start_sequence ' + ''.join(descriptions[image_id][i]) + ' end_sequence')
        train_descriptions[image_id] = train_descriptions_intermediate

In [16]:
train_descriptions

{'3432637363_3ba357e2da': ['start_sequence man in turban looks at two police officers who are near tent and pile of refuse end_sequence',
  'start_sequence men with hats on stand outside of white tent end_sequence',
  'start_sequence turbaned man and two police officers stand in front of white canopy and pile of garbage bags end_sequence',
  'start_sequence two officers with orange jackets stand outside white tent with onlookers end_sequence',
  'start_sequence two police officers observe while crowd gathers outside white tarp end_sequence'],
 '411175971_0fffd3b8c6': ['start_sequence brown dog is shown standing in the water near muddy beach end_sequence',
  'start_sequence dog happily plays in tidal overflow on cloudy day end_sequence',
  'start_sequence dog is playing in water on beach end_sequence',
  'start_sequence tan dog playing with stick on the shore end_sequence',
  'start_sequence the dog runs through the water with stick in its mouth end_sequence'],
 '2982928615_06db40f4cd':

In [18]:
############### already done. Dont execute again please #######################
f = open("flickr/Flickr8k_Dataset/descriptions.txt", "w+")
for key,value in train_descriptions.items():
    f.write(key)
    f.write("\t")
    f.write(str(value))
    f.write("\n")
f.close()

In [19]:
print(train_descriptions['3712923460_1b20ebb131'])
print(train_descriptions['2279980395_989d48ae72'])
print(len(train_descriptions))
#print(train_descriptions.keys())

['start_sequence bunch of people in camo pants run end_sequence', 'start_sequence group of men and woman wearing army pants and white shirt are running end_sequence', 'start_sequence group of men in camouflage pants and white tshirts are running in drill yard end_sequence', 'start_sequence group of people in army pants running end_sequence', 'start_sequence group of people wearing army clothes run together down the street end_sequence']
['start_sequence girl is buried under several playground balls end_sequence', 'start_sequence laughing baby is surrounded by balls of various colors end_sequence', 'start_sequence smiling child is surrounded by colored balls end_sequence', 'start_sequence little girl lying in many colored plastic balls end_sequence', 'start_sequence the child is covered with colorful balls end_sequence']
8092


In [16]:
############### already done. Dont execute again please #######################
model = InceptionV3(weights = 'imagenet')

W1028 09:11:47.621578 140026047493888 deprecation.py:237] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4139: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1028 09:12:08.404842 140026047493888 deprecation.py:506] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/training/moving_averages.py:211: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [17]:
############### already done. Dont execute again please #######################
model_new = Model(model.input, model.layers[-2].output)

In [18]:
############### already done. Dont execute again please #######################
model_new.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [46]:
############### already done. Dont execute again please #######################
path = glob.glob('flickr/Flickr8k_Dataset/Flicker8k_Dataset/*.jpg')
images = list()
count = 0
for img in path:
    
    #print(img)
    
    if count > 6000:
        n = cv2.imread(img)
    #print(n)
        x = image.img_to_array(n)
    #print(x)
        x = np.resize(x, (299, 299, 3))
        #print(x.shape)
        x = np.expand_dims(x, axis = 0)
    #print(x.shape)
        images.append(x)
    #x = x.reshape(1, 299, 299, 3)
    #x = np.expand_dims(x, axis = 3)
    #x = preprocess_input(x)
        count += 1
    else:
        count += 1
    

In [48]:
############### already done. Dont execute again please #######################
def image_to_vector(img):
    x = model_new.predict(img)
    feature_vector_2048 = np.reshape(x, x.shape[1])
    return feature_vector_2048 

In [49]:
############### already done. Dont execute again please #######################
fea_vec_2048 = []
for i in images:
    fea_vec_2048.append(image_to_vector(i))

In [51]:
############### already done. Dont execute again please #######################
with open("flickr/Flickr8k_Dataset/encoded_test_images.pkl", "wb") as encoded_pickle:
    pickle.dump(fea_vec_2048, encoded_pickle)

In [21]:
encoded_train_images = []
with open("flickr/Flickr8k_Dataset/encoded_train_images.pkl",'rb') as rfp: 
    while 1:
        try:
            encoded_train_images.append(pickle.load(rfp))
        except EOFError:
            break

In [22]:
encoded_train_images1 = []
with open("flickr/Flickr8k_Dataset/encoded_train_images1.pkl",'rb') as rfp1: 
    while 1:
        try:
            encoded_train_images1.append(pickle.load(rfp1))
        except EOFError:
            break

In [23]:
encoded_train_images.append(encoded_train_images1)

In [27]:
f = open("flickr/Flickr8k_Dataset/vocabulary1.txt", "r")
if f.mode == "r":
    vocabfromfile = f.readlines()
print(len(vocabfromfile))
f.close()

1839


In [28]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocabfromfile:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [29]:
print(ix)
print(len(ixtoword))
vocab_size = len(ixtoword) + 1

1840
1839


In [30]:
f = open("flickr/Flickr8k_Dataset/descriptions.txt", "r")
if f.mode == "r":
    descriptions = f.readlines()
print(len(descriptions))
f.close()


8092


In [31]:
def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
    lines = to_lines(descriptions)
    return max(len(d.split()) for d in lines)

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

Description Length: 34


In [32]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [33]:
######################### already done this. dont do this again please ################
glove_dir = 'glove.6B'
embeddings_index = {} # empty dictionary
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [34]:
embedding_dim = 200

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [35]:
embedding_matrix.shape

(1840, 200)

In [36]:
############ lstm model #########################3

inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

W1030 09:23:35.304345 140213276894976 deprecation.py:506] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1030 09:23:35.334423 140213276894976 deprecation.py:237] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4139: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [37]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [38]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

W1030 09:24:15.073977 140213276894976 deprecation.py:237] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [39]:
model.optimizer.lr = 0.0001
epochs = 10
number_pics_per_batch = 6
steps = len(train_descriptions)//number_pics_per_batch

In [ ]:
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)

In [ ]:
model.save("flickr/Flicker8k_Datset/model1.h5")